<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [50]</a>'.</span>

# 3. Building Compositional Space Complexes to Express Elaborate Design Problems

> ***(before running this code, please consult Quick Start to make sure everything is set up)***

> You will need the same dependencies as in [Tutorial No.2](02.AdditiveManufacturingPathPlanning.ipynb) (i.e., `pycalphad` and `pathfinding`), and additionally `igraph` in order to efficiently visualize the examples presented in this tutorial. **If you are running this in `nimplex`'s Codespaces, everything has been pre-installed for you.**

**In this tutorial, we will build upon `nimplex`'s graph representations and leverage our ability to combine ("stitch") them together without affecting the way we express the problem at hand and in an effortless fashion to dramatically speed up materials exploration.** In particular, we will show how to:

1. **Plot graphs** (as in mathematical objects, not any figures) using [`igraph`](https://igraph.org/), which can be done quite directly using `nimplex`'s point grid and neighbor lists `edges = [(i,n) for i in range(len(gridAtt)) for n in nList[i]]`

2. **Identify ordered subsystems in `nimplex` grids, like `A-B-C` and `C-A-B` within `A-B-C-D-E` and `D-C-G-F-A-H-B` to establish connectivity between them.** We will use this to combine 3 4-component systems (tetrahedra) by 2 3-component subsystems (triangles) to create a chain.

3. Identify all subspaces of a given order in low dimensional and high dimensional spaces (e.g., all quantized compositions of any 3 things out of N) and combine ("stitch") them together to form a **simplex graphs that intersect themselves in 3D because of high dimensionality, yet still have the same (graph) structure**.

4. **Construct a graph complex to explore all ternary combinations of `["Ti50Zr50", "Hf95Ti5", "NbTaWHf", "Mo80Nb10W10", "TiTa2", "Nb96Mo3W1", "Zr49 Hf1 Mo50"]` (7 alloys)** under equilibrium phase constraint (relatively expensive to compute) and then explore the space with an additional low-cost screenig constraint (RMSAD - alloy strenght surrogate) that could also be an ML surrogate.

In [53]:
import nimplex
from utils import stitching
from IPython.display import clear_output
from itertools import combinations

In [54]:
import igraph as ig
import plotly.graph_objs as go
import numpy as np
import random
random.seed(123)

## Plotting 4-Component Simplex Graph with `igraph`

In [55]:
dim = 4
ndiv = 6
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)

In [56]:
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [57]:
def plotGraph(edges: list):
    # Lets generate some seed positions for the nodes
    G = ig.Graph(edges)
    layout = G.layout_kamada_kawai(dim=3)
    layout_array = np.array(layout.coords)
    # Create edge traces
    edge_traces = []
    for edge in G.es:
        start, end = edge.tuple
        x0, y0, z0 = layout_array[start]
        x1, y1, z1 = layout_array[end]
        width = 5
        edge_trace = go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1],
                                line=dict(width=width, color='lightgray'),
                                opacity=0.2,
                                hoverinfo='none', mode='lines')
        edge_traces.append(edge_trace)

    node_trace = go.Scatter3d(x=layout_array[:, 0], y=layout_array[:, 1], z=layout_array[:, 2],
                            mode='markers',
                            marker=dict(size=3, 
                                        opacity=0.5,
                                        line=None,
                                        color='blue'),
                            
                            text=[f"Node {i}" for i in range(len(gridAtt))],
                            hoverinfo='text')

    # Create figure
    fig = go.Figure(data=edge_traces + [node_trace])

    # Update layout
    fig.update_layout(
        title='3D Graph with Feasibility Types',
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        showlegend=False,
        margin=dict(l=0, r=0, b=0, t=0),
        width=800
    )

    fig.show()

In [58]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 3 4-Component Systems into A Complex

In [59]:
sys1 = ["A", "B", "C", "D"]
sys2 = ["B", "C", "D", "E"]
sys3 = ["B", "D", "E", "F"]

In [60]:
offset = len(gridAtt)
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset,n+offset))
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset*2,n+offset*2))

In [61]:
#plotGraph(edges)
clear_output()

In [62]:
stitchingPoints = stitching.findStitchingPoints_py(dim, ndiv, components=["A", "B", "C", "D"])
print(stitchingPoints["A-B"])

[83, 82, 79, 73, 63, 48, 27]


In [63]:
stitchingPoints1 = stitching.findStitchingPoints_py(dim, ndiv, components=sys1, offset=0)
stitchingPoints2 = stitching.findStitchingPoints_py(dim, ndiv, components=sys2, offset=len(gridAtt))
stitchingPoints3 = stitching.findStitchingPoints_py(dim, ndiv, components=sys3, offset=len(gridAtt)*2)

In [64]:
for i, j in zip(stitchingPoints1["B-C-D"], stitchingPoints2["B-C-D"]):
    edges.append((i, j))

In [65]:
for i, j in zip(stitchingPoints2["B-D-E"], stitchingPoints3["B-D-E"]):
    edges.append((i, j))

In [66]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 9 3-Component Systems into A Complex
### Disregard this section for now Adam

In [67]:
dim = 3
ndiv = 6
nsys = 9
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [68]:
len(gridAtt)

28

In [69]:
offset = len(gridAtt)
for sys in range(1,nsys+1):
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset*sys,n+offset*sys))

In [70]:
#plotGraph(edges)
clear_output()

In [71]:
import pandas as pd

In [72]:
from pycalphad import Database

In [73]:
dbCrNiV = Database("ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB")
dbCrFeTi = Database("ammap/databases/Cr-Fe-Ti_wang2017.tdb")
dbCrFeNi = Database("ammap/databases/Cr-Fe-Ni_miettinen1999.tdb")
dbCrNiTi= Database("ammap/databases/Cr-Ni-Ti_huang2018.tdb")
dbCrTiV= Database("ammap/databases/Cr-Ti-V_ghosh2002.tdb")
dbFeNiTi= Database("ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb")
dbFeNiV= Database("ammap/databases/Fe-Ni-V_zhao2014.tdb")
dbFeTiV= Database("ammap/databases/Fe-Ti-V_guo2012.TDB")
dbNiTiV= Database("ammap/databases/Ni-Ti-V_zou2018.tdb")
CrNiV = ['Cr', 'Ni', 'V']
CrFeTi = ['Cr', 'Fe', 'Ti']
CrFeNi = ['Cr', 'Fe', 'Ni']
CrNiTi = ['Cr', 'Ni', 'Ti']
CrTiV = ['Cr', 'Ti', 'V']
FeNiTi = ['Fe', 'Ni', 'Ti']
FeNiV = ['Fe', 'Ni', 'V']
FeTiV = ['Fe', 'Ti', 'V']
NiTiV = ['Ni', 'Ti', 'V']


/ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/pycalphad/io/tdb.py:293: UserWarning: The type definition character `&` in `TYPE_DEFINITION & GES A_P_D A2_BCC MAGNETIC -1.0 4.00000E-01 ` is not used by any phase.
  warnings.warn(f"The type definition character `{typechar}` in `TYPE_DEFINITION {typechar} {line}` is not used by any phase.")


In [74]:
stitchingPointsCrNiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiV, offset=0)
stitchingPointsCrFeTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeTi, offset=len(gridAtt))
stitchingPointsCrFeNi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeNi, offset=len(gridAtt)*2)
stitchingPointsCrNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiTi, offset=len(gridAtt)*3)
stitchingPointsCrTiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrTiV, offset=len(gridAtt)*4)
stitchingPointsFeNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiTi, offset=len(gridAtt)*5)
stitchingPointsFeNiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiV, offset=len(gridAtt)*6)
stitchingPointsFeTiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeTiV, offset=len(gridAtt)*7)
stitchingPointsNiTiV = stitching.findStitchingPoints_py(dim, ndiv, components=NiTiV, offset=len(gridAtt)*8)

## Finding and Combining 3-Component Subspaces in 4-Component Space

In [75]:
from itertools import combinations
elements = ["A", "B", "C", "D"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]


In [76]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [77]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))

In [78]:
#plotGraph(edges)
clear_output()

In [79]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1)]
('A', 'C'): occurs between ternary [(0, 2)]
('B', 'C'): occurs between ternary [(0, 3)]
('A', 'D'): occurs between ternary [(1, 2)]
('B', 'D'): occurs between ternary [(1, 3)]
('C', 'D'): occurs between ternary [(2, 3)]


In [80]:
for stitchingBinary, ternaryPair in stitchingBinaries.items():
    ternary1, ternary2 = ternaryPair[0][0], ternaryPair[0][1]
    stitching1 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary1],
        offset=ternary1*len(gridAtt)
        )["-".join(stitchingBinary)]
    stitching2 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary2],
        offset=ternary2*len(gridAtt)
        )["-".join(stitchingBinary)]
    print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
    for i, j in zip(stitching1, stitching2):
        edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('B', 'C') from [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 1 and 2 at ('A', 'D') from [181, 179, 176, 172, 167, 161, 154, 146, 137, 127, 116, 104, 91] to [272, 270, 267, 263, 258, 252, 245, 237, 228, 218, 207, 195, 182]
Stitching 1 and 3 at ('B', 'D') from [103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91] to [363, 361, 358, 354, 349, 343, 336, 328, 319, 309, 298, 286, 273]
Stitching 2 and 3 at ('C', 'D') from [194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182] to [285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273]


In [81]:
#plotGraph(edges)
clear_output()

## In Higher Dimensional Cases (Stretched 2D Spaces Crossing Each Other in 3D)

In [82]:
from itertools import combinations
elements = ["A", "B", "C", "D", "E"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'B', 'E'), ('A', 'C', 'D'), ('A', 'C', 'E'), ('A', 'D', 'E'), ('B', 'C', 'D'), ('B', 'C', 'E'), ('B', 'D', 'E'), ('C', 'D', 'E')]


In [83]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [84]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))


In [85]:
#plotGraph(edges)
clear_output()

In [86]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('A', 'C'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('B', 'C'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('A', 'D'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('B', 'D'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('A', 'E'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('B', 'E'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('C', 'D'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('C', 'E'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('D', 'E'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [87]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 1 and 2 at ('A', 'B') from [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 0 and 4 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]
Stitching 3 and 4 at ('A', 'C') from [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]
Stitching 0 and 6 at (

In [88]:
#plotGraph(edges)
clear_output()

## Exploring Complex Formed by All 3-Component Alloy Subsystems of 5-Component Space (w. Infeasibility Gliding) #THIS ONE ADAM

In [89]:
elementalSpaceComponents = ["Cr","Fe","Ni","Ti","V"]
#attainableSpaceComponents = ["SS304L", "Ni", "Cr", "V", "Ti"]
attainableSpaceComponents = ["Cr", "Fe", "Ni", "Ti", "V"]
attainableSpaceComponentPositions = [
    #"Cr","Fe","Ni","Ti","V"
    #[0.1993865031, 0.7044989775,0.096114519430,0,0],
    [1,0,0,0,0],
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,0,1]
]    
ternaries = list(combinations(attainableSpaceComponents, 3))
ternaries_CompPos = list(combinations(attainableSpaceComponentPositions, 3))
ndiv = 8
gridAtt, nList = nimplex.simplex_graph_py(3, ndiv)

for tern, terncp in zip(ternaries, ternaries_CompPos):
    print(f"{str(tern):<40} -> {terncp}")

('Cr', 'Fe', 'Ni')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0])
('Cr', 'Fe', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Fe', 'V')                        -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ni', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Ni', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ti', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Fe', 'Ni', 'Ti')                       -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Fe', 'Ni', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Fe', 'Ti', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Ni', 'Ti', 'V')                        -> ([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])


In [90]:
# Edges list for graph plotting and path finding purposes
edges = []
# Connectivity list within each subsystem
graphN = [[] for i in range(len(gridAtt * len(ternaries)))]
# Connectivity list between subsystems
graphNS = [[] for i in range(len(graphN))]
compositions = []
compositions_with_id = []  # List to store compositions with their identifiers
ternaries_with_id = []  # New list to store ternaries_CompPos with their identifiers

# Iterate over ternaries
for i, terncp in enumerate(ternaries_CompPos):
    ternaries_with_id.append((terncp, i))  # Add terncp and its id to the new list
    
    offset = i*len(gridAtt)
    for j in range(len(gridAtt)):
        for n in nList[j]:
            edges.append((j+offset,n+offset))
            graphN[j+offset].append(n+offset)
    gridAttTemp, gridElTemp = nimplex.embeddedpair_simplex_grid_fractional_py(terncp, ndiv)
    compositions += gridElTemp
    
    # Attach identifier to each composition
    compositions_with_id.extend([(comp, i) for comp in gridElTemp])

In [91]:
# Example: Access ternaries_CompPos with their identifiers
for terncp, ternary_id in ternaries_with_id:
    print(f"Ternary composition position {terncp} has id {ternary_id}")

Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0]) has id 0
Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0]) has id 1
Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]) has id 2
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]) has id 3
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]) has id 4
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 5
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]) has id 6
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]) has id 7
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 8
Ternary composition position ([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 9


In [92]:
mapping = {}
for ternary, id in ternaries_with_id:
    composition_key = ''.join(elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary))))
    mapping[composition_key] = id

In [93]:
mapping = {}
for ternary, id in ternaries_with_id:
    composition_key = ''.join(elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary))))
    individual_elements = [elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary)))]
    mapping[composition_key] = {
        'id': id,
        'elements': individual_elements
    }

# Print the mapping to see the result
for key, value in mapping.items():
    print(f"Combination: {key}, ID: {value['id']}, Elements: {value['elements']}")

Combination: CrFeNi, ID: 0, Elements: ['Cr', 'Fe', 'Ni']
Combination: CrFeTi, ID: 1, Elements: ['Cr', 'Fe', 'Ti']
Combination: CrFeV, ID: 2, Elements: ['Cr', 'Fe', 'V']
Combination: CrNiTi, ID: 3, Elements: ['Cr', 'Ni', 'Ti']
Combination: CrNiV, ID: 4, Elements: ['Cr', 'Ni', 'V']
Combination: CrTiV, ID: 5, Elements: ['Cr', 'Ti', 'V']
Combination: FeNiTi, ID: 6, Elements: ['Fe', 'Ni', 'Ti']
Combination: FeNiV, ID: 7, Elements: ['Fe', 'Ni', 'V']
Combination: FeTiV, ID: 8, Elements: ['Fe', 'Ti', 'V']
Combination: NiTiV, ID: 9, Elements: ['Ni', 'Ti', 'V']


In [94]:
mapping

{'CrFeNi': {'id': 0, 'elements': ['Cr', 'Fe', 'Ni']},
 'CrFeTi': {'id': 1, 'elements': ['Cr', 'Fe', 'Ti']},
 'CrFeV': {'id': 2, 'elements': ['Cr', 'Fe', 'V']},
 'CrNiTi': {'id': 3, 'elements': ['Cr', 'Ni', 'Ti']},
 'CrNiV': {'id': 4, 'elements': ['Cr', 'Ni', 'V']},
 'CrTiV': {'id': 5, 'elements': ['Cr', 'Ti', 'V']},
 'FeNiTi': {'id': 6, 'elements': ['Fe', 'Ni', 'Ti']},
 'FeNiV': {'id': 7, 'elements': ['Fe', 'Ni', 'V']},
 'FeTiV': {'id': 8, 'elements': ['Fe', 'Ti', 'V']},
 'NiTiV': {'id': 9, 'elements': ['Ni', 'Ti', 'V']}}

In [95]:
# def find_duplicates(lst):
#     duplicates_counts = {}
#     for item in lst:
#         # Use the string representation for complex/unhashable types
#         item_str = str(item)
#         if item_str in duplicates_counts:
#             duplicates_counts[item_str] += 1
#         else:
#             duplicates_counts[item_str] = 1

#     # Filter out items that only appear once
#     duplicates = {item: count for item, count in duplicates_counts.items() if count > 1}
    
#     if not duplicates:
#         return "No duplicates found in the list."
#     else:
#         result = "Duplicates found:"
#         for item, count in duplicates.items():
#             result += f"{item} appears {count} times"
#         return result.strip()

In [96]:
# find_duplicates(compositions)

In [97]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('Cr', 'Fe'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('Cr', 'Ni'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('Fe', 'Ni'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('Cr', 'Ti'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('Fe', 'Ti'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('Cr', 'V'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('Fe', 'V'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('Ni', 'Ti'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('Ni', 'V'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('Ti', 'V'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [98]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))
            graphNS[i].append(j)

Stitching 0 and 1 at ('Cr', 'Fe') from [44, 43, 41, 38, 34, 29, 23, 16, 8] to [89, 88, 86, 83, 79, 74, 68, 61, 53]
Stitching 0 and 2 at ('Cr', 'Fe') from [44, 43, 41, 38, 34, 29, 23, 16, 8] to [134, 133, 131, 128, 124, 119, 113, 106, 98]
Stitching 1 and 2 at ('Cr', 'Fe') from [89, 88, 86, 83, 79, 74, 68, 61, 53] to [134, 133, 131, 128, 124, 119, 113, 106, 98]
Stitching 0 and 3 at ('Cr', 'Ni') from [44, 42, 39, 35, 30, 24, 17, 9, 0] to [179, 178, 176, 173, 169, 164, 158, 151, 143]
Stitching 0 and 4 at ('Cr', 'Ni') from [44, 42, 39, 35, 30, 24, 17, 9, 0] to [224, 223, 221, 218, 214, 209, 203, 196, 188]
Stitching 3 and 4 at ('Cr', 'Ni') from [179, 178, 176, 173, 169, 164, 158, 151, 143] to [224, 223, 221, 218, 214, 209, 203, 196, 188]
Stitching 0 and 6 at ('Fe', 'Ni') from [8, 7, 6, 5, 4, 3, 2, 1, 0] to [314, 313, 311, 308, 304, 299, 293, 286, 278]
Stitching 0 and 7 at ('Fe', 'Ni') from [8, 7, 6, 5, 4, 3, 2, 1, 0] to [359, 358, 356, 353, 349, 344, 338, 331, 323]
Stitching 6 and 7 at ('Fe'

In [99]:
from pycalphad import Database

# List of database file paths
db_files = [
    "ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB",
    "ammap/databases/Cr-Fe-Ti_wang2017.tdb",
    "ammap/databases/Cr-Fe-Ni_miettinen1999.tdb",
    "ammap/databases/Cr-Ni-Ti_huang2018.tdb",
    "ammap/databases/Cr-Ti-V_ghosh2002.tdb",
    "ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb",
    "ammap/databases/Fe-Ni-V_zhao2014.tdb",
    "ammap/databases/Fe-Ti-V_guo2012.TDB",
    "ammap/databases/Ni-Ti-V_zou2018.tdb"
]

# Dictionary to store unique phases for each database
unique_phases = {}

# Iterate through each database file
for db_file in db_files:
    dbf = Database(db_file)
    phases = list(set(dbf.phases.keys()))
    unique_phases[db_file] = phases

# Print unique phases for each database
for db_file, phases in unique_phases.items():
    print(f"Unique phases for {db_file}: {phases}")

Unique phases for ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB: ['HIGH_SIGMA', 'HCP_A3', 'FCC_A1', 'M3V', 'BCC_A2', 'L12_FCC', 'NI2V', 'B2_BCC', 'NI2V7', 'SIGMA', 'LIQUID', 'COV3']
Unique phases for ammap/databases/Cr-Fe-Ti_wang2017.tdb: ['C15', 'HCP_A3', 'FCC_A1', 'C14', 'BCC_A2', 'C36', 'SIGMA', 'BCC_B2', 'LIQUID', 'TI5CR7FE17']
Unique phases for ammap/databases/Cr-Fe-Ni_miettinen1999.tdb: ['HCP_A3', 'FCC_A1', 'BCC_A2', 'SIGMA', 'LIQUID']
Unique phases for ammap/databases/Cr-Ni-Ti_huang2018.tdb: ['NITI2', 'HCP_A3', 'FCC_A1', 'NI3TI', 'LAVES_C36', 'BCC_A2', 'LAVES_C14', 'NITI', 'LAVES_C15', 'LIQUID']
Unique phases for ammap/databases/Cr-Ti-V_ghosh2002.tdb: ['HCP_A3', 'LAVES_C36', 'BCC_A2', 'LAVES_C14', 'LAVES_C15', 'LIQUID']
Unique phases for ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb: ['NITI2', 'C14', 'A2', 'NI3TI', 'FCC4', 'A1', 'LIQUID', 'BCC2', 'A3']
Unique phases for ammap/databases/Fe-Ni-V_zhao2014.tdb: ['HCP_A3', 'FCC_A1', 'D022_NI3V', 'CBCC_A12', 'BCC_A2', 'NI2V', 'A2_BCC', 'LAVE

In [100]:

#dbf = Database("CrHfMoNbTaTiVWZr_9element_Feb2023.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')
#from myPycalphadCallable import equilibrium_callable

['Cr', 'Fe', 'Ni', 'Ti', 'V']
Loaded TDB file with phases considered: ['B2', 'NITI2', 'HCP_A3', 'FCC_A1', 'NI3TI', 'NI3V', 'CBCC_A12', 'BCC_A2', 'NI2V', 'BCT_A5', 'DIAMOND_A4', 'LAVES_C14', 'SIGMA', 'LAVES_C15', 'LIQUID', 'NIV3', 'CUB_A13']


In [101]:
import os
import importlib

# Directory containing the equilibrium files
directory = "ammap/callables/multi_system_equilibrium_and_scheil"

# Get all files starting with "equilibrium"
equilibrium_files = [f for f in os.listdir(directory) if f.startswith("equilibrium") and f.endswith(".py")]

# Dictionary to store imported callables with unique names
equilibrium_callables = {}

# Import each equilibrium file and store the callable with a unique name
for file in equilibrium_files:
    module_name = file[:-3]  # Remove the .py extension
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    module = importlib.import_module(module_path)
    callable_name = f"{module_name}"
    equilibrium_callables[callable_name] = getattr(module, "equilibrium_callable")

# Print the imported callables
for name, func in equilibrium_callables.items():
    print(f"Imported {name}: {func}")

Imported equilibrium_callable_CrFeTi_08bbfb9a: <function equilibrium_callable at 0x7f0ee141e480>
Imported equilibrium_callable_CrFeV_ab1edb07: <function equilibrium_callable at 0x7f0ed8ca5440>
Imported equilibrium_callable_CrNiTi_a9f6f2ff: <function equilibrium_callable at 0x7f0ed8a49120>
Imported equilibrium_callable_NiTiV_1d83c99c: <function equilibrium_callable at 0x7f0ed8880ae0>
Imported equilibrium_callable_NiCrV_b7aba9ab: <function equilibrium_callable at 0x7f0ee1551f80>
Imported equilibrium_callable_NiCrFe_f434a6d9: <function equilibrium_callable at 0x7f0ed81062a0>
Imported equilibrium_callable_FeTiV_fa95b3ee: <function equilibrium_callable at 0x7f0ed7f44e00>
Imported equilibrium_callable_FeNiV_b9b0384d: <function equilibrium_callable at 0x7f0ed8457e20>
Imported equilibrium_callable_CrTiV_ed4c332b: <function equilibrium_callable at 0x7f0ed79ac360>
Imported equilibrium_callable_FeNiTi_06a49695: <function equilibrium_callable at 0x7f0ed782b380>


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [102]:
# Get all files starting with "scheil"
scheil_files = [f for f in os.listdir(directory) if f.startswith("scheil") and f.endswith(".py")]
scheil_callables = {}

In [103]:
print(scheil_files)
print(equilibrium_files)
print(equilibrium_callables)

['scheil_callable_FeNiTi_06a49695.py', 'scheil_callable_NiCrFe_f434a6d9.py', 'scheil_callable_CrNiTi_a9f6f2ff.py', 'scheil_callable_FeNiV_b9b0384d.py', 'scheil_callable_CrFeV_ab1edb07.py', 'scheil_callable_NiCrV_b7aba9ab.py', 'scheil_callable_FeTiV_fa95b3ee.py', 'scheil_callable_CrTiV_ed4c332b.py', 'scheil_callable_NiTiV_1d83c99c.py', 'scheil_callable_CrFeTi_08bbfb9a.py']
['equilibrium_callable_CrFeTi_08bbfb9a.py', 'equilibrium_callable_CrFeV_ab1edb07.py', 'equilibrium_callable_CrNiTi_a9f6f2ff.py', 'equilibrium_callable_NiTiV_1d83c99c.py', 'equilibrium_callable_NiCrV_b7aba9ab.py', 'equilibrium_callable_NiCrFe_f434a6d9.py', 'equilibrium_callable_FeTiV_fa95b3ee.py', 'equilibrium_callable_FeNiV_b9b0384d.py', 'equilibrium_callable_CrTiV_ed4c332b.py', 'equilibrium_callable_FeNiTi_06a49695.py']
{'equilibrium_callable_CrFeTi_08bbfb9a': <function equilibrium_callable at 0x7f0ee141e480>, 'equilibrium_callable_CrFeV_ab1edb07': <function equilibrium_callable at 0x7f0ed8ca5440>, 'equilibrium_calla

In [104]:
import importlib
import os

# Debugging the scheil_files loop
for file in scheil_files:
    module_name = file[:-3]  # Remove the .py extension
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    print(f"Importing module: {module_path}")
    try:
        module = importlib.import_module(module_path)
        callable_name = f"{module_name}"
        scheil_callables[callable_name] = getattr(module, "scheil_callable")
        print(f"Added scheil callable: {callable_name}")
    except Exception as e:
        print(f"Failed to import {module_path}: {e}")

for name, func in scheil_callables.items():
    print(f"Imported {name}: {func}")


Importing module: ammap.callables.multi_system_equilibrium_and_scheil.scheil_callable_FeNiTi_06a49695
Failed to import ammap.callables.multi_system_equilibrium_and_scheil.scheil_callable_FeNiTi_06a49695: /lib64/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/scipy/spatial/_ckdtree.cpython-311-x86_64-linux-gnu.so)
Importing module: ammap.callables.multi_system_equilibrium_and_scheil.scheil_callable_NiCrFe_f434a6d9
Failed to import ammap.callables.multi_system_equilibrium_and_scheil.scheil_callable_NiCrFe_f434a6d9: /lib64/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/scipy/spatial/_ckdtree.cpython-311-x86_64-linux-gnu.so)
Importing module: ammap.callables.multi_system_equilibrium_and_scheil.scheil_callable_CrNiTi_a9f6f2ff
Failed to import ammap.callables.multi_system_equilibrium_

In [ ]:
def process_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        middle_part = parts[2]
        if middle_part in element_mapping:
            return middle_part, element_mapping[middle_part]
    return None, None

In [ ]:
element_mapping=mapping

In [ ]:
def process_key(key):
    parts = key.split('_')
    if len(parts) >= 3:
        middle_part = parts[2]
        if middle_part in mapping:
            return middle_part, mapping[middle_part]['id'], mapping[middle_part]['elements']
        
        # If direct matching fails, try matching by elements
        middle_elements = set(middle_part[i:i+2] for i in range(0, len(middle_part), 2))
        for map_key, value in mapping.items():
            if set(value['elements']) == middle_elements:
                return middle_part, value['id'], value['elements']
    
    return None, None, None

# Process each key in the equilibrium_callables dictionary
id_to_callable = {}
for key in equilibrium_callables:
    middle_part, mapping_id, elements = process_key(key)
    if middle_part and mapping_id is not None:
        print(f"Key: {key}")
        print(f"The mapping number for {middle_part} is {mapping_id}")
        print(f"Elements: {elements}")
        print("---")
        id_to_callable[mapping_id]=key
    else:
        print(f"No mapping found for key: {key}")
        print("---")

In [ ]:
id_to_callable

In [ ]:
compositions_with_id[:5]

In [ ]:
mapping

In [ ]:
def reduce_compositions(compositions_with_id, mapping):
    element_order = elementalSpaceComponents
    system_comps_with_id = []
    reduced_compositions = []
    
    for composition, comp_id in compositions_with_id:
        relevant_entry = next((entry for entry in mapping.values() if entry['id'] == comp_id), None)
        if not relevant_entry:
            continue
        
        relevant_indices = [element_order.index(elem) for elem in relevant_entry['elements']]
        reduced_point = [composition[index] for index in relevant_indices]
        system_comps_with_id.append((reduced_point, comp_id))
        reduced_compositions.append(reduced_point)
    
    return system_comps_with_id, reduced_compositions

In [ ]:
system_comps_with_id, reduced_compositions = reduce_compositions(compositions_with_id, mapping)

In [ ]:
from tqdm.contrib.concurrent import process_map

In [ ]:
startingNodes = [0, 90, 200, 310] + random.sample(range(len(reduced_compositions)), 11)
print(f"Starting nodes: {startingNodes}")

for startingNode in startingNodes:
    print(f"Starting node: {reduced_compositions[startingNode]}")

In [ ]:
gridFeasible = [None]*len(reduced_compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

In [ ]:
# Function to get the correct callable for a given composition
def get_callable(composition):
    for comp, id in compositions_with_id:
        if comp == composition:
            callable_name = id_to_callable.get(id)
            if callable_name is None:
                raise ValueError(f"No callable name found for ID {id}")
            if callable_name in globals():
                return globals()[callable_name]
            else:
                raise NameError(f"Function '{callable_name}' not found in global scope")
    raise ValueError(f"No callable found for composition {composition}")

def process_composition(elP):
    try:
        callable_func = get_callable(elP)
        return callable_func(elP)
    except Exception as e:
        print(f"Error processing composition {elP}: {str(e)}")
        return None


In [ ]:
equilibrium_callables

In [ ]:
id_to_callable

In [ ]:
sampled_compositions = compositions_with_id[::50]

# Print the sampled compositions to verify
for composition in sampled_compositions:
    print(composition)

In [ ]:
print(equilibrium_callables['equilibrium_callable_CrNiTi_a9f6f2ff'](compositions[200]))
compositions[200]

In [ ]:
from functools import partial

def get_equilibrium_callable(composition, id_to_callable, equilibrium_callables):
    composition_id = composition[1]  # Get the ID from the composition tuple
    #print(f"Composition ID: {composition_id}")
    callable_name = id_to_callable.get(composition_id)
    if callable_name is None:
        raise ValueError(f"No callable found for composition ID {composition_id}")
    callable_func = equilibrium_callables.get(callable_name)
    if callable_func is None:
        raise ValueError(f"No callable function found for name {callable_name}")
    return callable_func

def apply_equilibrium_callable(callable_and_position):
    callable_func, position = callable_and_position
    return callable_func(position)

In [ ]:
id_to_callable

In [ ]:
from functools import partial

def get_scheil_callable(composition, id_to_callable, scheil_callables):
    composition_id = composition[1]  # Get the ID from the composition tuple
    #print(f"Composition ID: {composition_id}")
    callable_name = id_to_callable.get(composition_id)
    if callable_name is None:
        raise ValueError(f"No callable found for composition ID {composition_id}")
    callable_func = scheil_callables.get(callable_name)
    if callable_func is None:
        raise ValueError(f"No callable function found for name {callable_name}")
    return callable_func

def apply_scheil_callable(callable_and_position):
    callable_func, position = callable_and_position
    return callable_func(position)

In [ ]:
while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of equilibrium callables for each composition
    equilibrium_callables_list = [get_equilibrium_callable(system_comps_with_id[i], id_to_callable, equilibrium_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_equilibrium_callable, zip(equilibrium_callables_list, elPositions), max_workers=4)
    else:
        results = [ec(elP) for ec, elP in zip(equilibrium_callables_list, elPositions)]
    
    # Extract only the 'Phases' component from the results
    phases = [result['Phases'] for result in results]
    
    feasibilities = [len(set(p) & set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC','A2_FCC'])) == 0 and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

Out of the explored points, around 1040 should be feasible (i.e., satisfy the equilibrium constraint).

## Adding More Constraints

In [ ]:
import pqam_rmsadtandoc2023

def c2rmsad(point):
    formula = ' '.join([f'{c}{p}' for c, p in zip(elementalSpaceComponents, point)])
    return pqam_rmsadtandoc2023.predict(formula)

In [ ]:
print(compositions[0])
c2rmsad(compositions[0])

In [ ]:
avgRMSAD = sum([c2rmsad(p) for p in compositions])/len(compositions)
print(f"Average RMSAD: {avgRMSAD}")

In [ ]:
startingNodes = [0, 90, 742, 799, 1048, 1070, 1538] + random.sample(range(len(compositions)), 13)

for startingNode in startingNodes:
    print(f"Starting node ({startingNode}): {compositions[startingNode]}")

In [ ]:
gridFeasible = [None]*len(compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

minRMSAD = 0.2
print(f"Minimum RMSAD: {minRMSAD}")

In [ ]:
while len(queue)>0:
    print(f"Queue: {queue}")
    # Skip all points with RMSAD below the minimum (inexpensive) by removing them from the queue, marking them as infeasible, and adding them to explored
    for i in queue:
        if c2rmsad(compositions[i]) < minRMSAD:
            for eq in [i] + graphNS[i]:
                gridFeasible[eq] = False
            explored = explored.union([i] + graphNS[i])
    queue = [i for i in queue if gridFeasible[i] is None]

    # Calculate CALPHAD feasibilities (expensive) of the current queue
    elPositions = [compositions[i] for i in queue]
    if len(queue)>3:
        phases = process_map(equilibrium_callable, elPositions, max_workers=4)
    else:
        phases = [equilibrium_callable(elP) for elP in elPositions]
    feasibilities = [len(set(p) & set(['LAVES_C15', 'LAVES_C36', 'LAVES_C14', 'LIQUID']))==0 and p!=[] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

In [ ]:
sum([f for f in gridFeasible if f])

In [ ]:
len(gridFeasible)